In [0]:
df_silver = spark.table("nttdataeducacao.silver.enade_2024")


In [0]:
from pyspark.sql import functions as F

df_gold_curso = (
    df_silver
    .groupBy(
        "codigo_do_curso",
        "nome_da_ies",
        "sigla_da_uf",
        "area_de_avaliacao",
        "modalidade_de_ensino"
    )
    .agg(
        F.count("*").alias("qtd_alunos"),
        F.avg("nota_bruta_fg").alias("media_fg"),
        F.avg("nota_bruta_ce").alias("media_ce"),
        F.avg("conceito_enade_continuo").alias("media_enade_continuo"),
        F.avg("nota_padronizada_fg").alias("media_fg_pad"),
        F.avg("nota_padronizada_ce").alias("media_ce_pad")
    )
)

df_gold_curso.write.format("delta").mode("overwrite").saveAsTable(
    "nttdataeducacao.gold.enade_curso"
)
display(df_gold_curso)

In [0]:
from pyspark.sql import functions as F

df_gold_area = (
    df_silver
    .groupBy("area_de_avaliacao")
    .agg(
        F.count("*").alias("qtd_cursos"),
        F.avg("nota_bruta_fg").alias("media_fg"),
        F.avg("nota_bruta_ce").alias("media_ce"),
        F.avg("conceito_enade_continuo").alias("media_enade_continuo")
    )
)

df_gold_area = (
    df_gold_area
    .withColumn("media_fg", F.round("media_fg", 1))
    .withColumn("media_ce", F.round("media_ce", 1))
    .withColumn("media_enade_continuo", F.round("media_enade_continuo", 1))
)

df_gold_area.write.format("delta").mode("overwrite").saveAsTable(
    "nttdataeducacao.gold.enade_area"
)

display(df_gold_area)


In [0]:
from pyspark.sql import functions as F

df_gold_uf = (
    df_silver
    .groupBy("sigla_da_uf")
    .agg(
        F.count("*").alias("qtd_cursos"),
        F.avg("conceito_enade_continuo").alias("media_enade")
    )
)

df_gold_uf = df_gold_uf.withColumn("media_enade", F.round("media_enade", 1))

df_gold_uf.write.format("delta").mode("overwrite").saveAsTable(
    "nttdataeducacao.gold.enade_uf"
)

display(df_gold_uf)
